In [125]:
# Import libraries

import warnings
import pandas as pd
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

In [126]:
data = pd.read_csv('data_dummy.csv')

In [127]:
# dummy dummy E-MINI ~ dummy FTSE
len1 = len(data['dummy E-MINI'])

X = data['dummy FTSE']
X = X.to_frame()

Y = data['dummy E-MINI']
Y = Y.to_frame()

warnings.filterwarnings('ignore')

X1 = X
print('Моdel: dummy E-MINI ~ dummy FTSE')
for i in range(0,11):
    if i == 0:
        Y = Y[:]
        X1 = X[:]
        ar1 = Y.to_numpy()
        ar2 = X1.to_numpy()

    else:
        Y1 = Y[i:]
        X1 = X1[1:]
        Xnew = X['dummy FTSE'][:len1-i].tolist()
        name = 'Lag '+str(i)
        X1[name] = Xnew
        ar1 = Y1.to_numpy()
        ar2 = X1.to_numpy()



    rf = RandomForestClassifier(n_estimators=500, max_features=0.25, criterion="entropy", class_weight="balanced")
    pip_baseline = make_pipeline(RobustScaler(), rf)
    results = cross_val_score(pip_baseline, ar2, ar1,
                         scoring="roc_auc", cv=5)
    print('Model №'+str(i)+":  Mean Roc_Auc = "+str(results.mean())+ '  lag = '+str(i))
    if results.mean() > max_ra:
        max_ra = results.mean()
        lag = i
print('Best model: ' + 'lag = '+str(lag) + '  Max mean Roc_Auc = ' + str(max_ra))

Моdel: dummy E-MINI ~ dummy FTSE
Model №0:  Mean Roc_Auc = 0.6583450043107757  lag = 0


KeyboardInterrupt: 

In [123]:
# Парные модели

dep = ['dummy E-MINI','dummy E-MINI','dummy FTSE','dummy FTSE','dummy TREASURY','dummy TREASURY']
exog = ['dummy FTSE','dummy TREASURY','dummy E-MINI','dummy TREASURY','dummy E-MINI','dummy FTSE']

best_pair_models = []


for j in range(len(dep)):

    len1 = len(data[exog[j]])

    X = data[exog[j]]
    X = X.to_frame()
    X = X.truncate(before=200000, after=250000)
    Y = data[dep[j]]
    Y = Y.to_frame()
    Y = Y.truncate(before=200000, after=250000)
    max_ra = 0
    warnings.filterwarnings('ignore')

    print('Моdel: '+dep[j]+ ' ~ '+ exog[j])

    for i in range(0,11):
        if i == 0:
            Y1 = Y[:]
            X1 = X[:]
            ar1 = Y1.to_numpy()
            ar2 = X1.to_numpy()

        else:
            Y1 = Y1[1:]
            X1 = X1[1:]
            Xi = X[:-i]
            Xi = Xi[exog[j]].tolist()
            X1['lag'+str(i)] = Xi
            ar1 = Y1.to_numpy()
            ar2 = X1.to_numpy()

        rf = RandomForestClassifier(n_estimators=500, max_features=0.25, criterion="entropy", class_weight="balanced")
        pip_baseline = make_pipeline(RobustScaler(), rf)
        results = cross_val_score(pip_baseline, ar2, ar1,
                             scoring="roc_auc", cv=5)
        print('Model №'+str(i)+":  Mean Roc_Auc = "+str(results.mean())+ '  lag = '+str(i))
        if results.mean() > max_ra:
            max_ra = results.mean()
            lag = i
    print('Best model: ' + 'lag = '+str(lag) + '  Max mean Roc_Auc = ' + str(max_ra))
    best_pair_models.append('Моdel: '+dep[j] + ' ~ '+ exog[j] + '    lag = '+str(lag) + '  Max mean Roc_Auc = ' + str(max_ra))



Моdel: dummy E-MINI ~ dummy  FTSE
Model №0:  Mean Roc_Auc = 0.6405862197502515  lag = 0
Model №1:  Mean Roc_Auc = 0.6443580572229054  lag = 1
Model №2:  Mean Roc_Auc = 0.6444352639093713  lag = 2
Model №3:  Mean Roc_Auc = 0.6442552224642308  lag = 3
Model №4:  Mean Roc_Auc = 0.6416763179551122  lag = 4
Model №5:  Mean Roc_Auc = 0.643008563545252  lag = 5
Model №6:  Mean Roc_Auc = 0.6423852200894562  lag = 6
Model №7:  Mean Roc_Auc = 0.6423811504115665  lag = 7
Model №8:  Mean Roc_Auc = 0.6438471942172599  lag = 8
Model №9:  Mean Roc_Auc = 0.6392602950524776  lag = 9
Model №10:  Mean Roc_Auc = 0.6289167652613579  lag = 10
Best model: lag = 2  Max mean Roc_Auc = 0.6444352639093713
Моdel: dummy E-MINI ~ dummy  TREASURY
Model №0:  Mean Roc_Auc = 0.5276524706455195  lag = 0
Model №1:  Mean Roc_Auc = 0.5267683568964435  lag = 1
Model №2:  Mean Roc_Auc = 0.5290808788015585  lag = 2
Model №3:  Mean Roc_Auc = 0.5321923935136412  lag = 3
Model №4:  Mean Roc_Auc = 0.5307949173201568  lag = 4
Mode

In [124]:
for q in best_pair_models:
    print(q)

Моdel: dummy E-MINI ~ dummy  FTSE    lag = 2  Max mean Roc_Auc = 0.6444352639093713
Моdel: dummy E-MINI ~ dummy  TREASURY    lag = 3  Max mean Roc_Auc = 0.5321923935136412
Моdel: dummy  FTSE ~ dummy E-MINI    lag = 3  Max mean Roc_Auc = 0.6461761302562574
Моdel: dummy  FTSE ~ dummy  TREASURY    lag = 1  Max mean Roc_Auc = 0.5213119553099486
Моdel: dummy  TREASURY ~ dummy E-MINI    lag = 4  Max mean Roc_Auc = 0.547137962142989
Моdel: dummy  TREASURY ~ dummy  FTSE    lag = 1  Max mean Roc_Auc = 0.5315328149777763


In [129]:
# Model: dummy E-MINI ~ dummy FTSE + dummy TREASURY

dep = ['dummy E-MINI']
exog = ['dummy FTSE','dummy TREASURY']
best_models3 = '0'
lag1 = [0,1,2,3,4,5,6,7,8,9,10]
lag2 = [0,1,2,3,4,5,6,7,8,9,10]

len1 = len(data[dep[0]])

X1 = data[exog[0]]
X1 = X1.to_frame()
X1 = X1.truncate(before=200000, after=250000)
X2 = data[exog[1]]
X2 = X2.to_frame()
X2 = X2.truncate(before=200000, after=250000)
Y = data[dep[0]]
Y = Y.to_frame()
Y = Y.truncate(before=200000, after=250000)
max_ra = 0

warnings.filterwarnings('ignore')

for i in range(len(lag1)):
    X1_copy = X1.copy()
    X2_copy = X2.copy()
    Y_copy = Y.copy()

    for k in range(len(lag2)):

        if lag2[k] >= lag1[i]:

            maxlag = lag2[k]


            Y_m = Y_copy[maxlag:]
            len2 = len(Y_m)

            X0 = X1_copy[maxlag - 0 : maxlag - 0 + len2]

            for lg1 in range(1,lag1[i]+1):
                Xi = X1_copy[maxlag - lg1 : maxlag - lg1 + len2]
                Xi  = Xi['dummy FTSE'].tolist()
                X0['lag1-'+str(lg1)] = Xi

            for lg2 in range(0,maxlag+1):
                Ai = X2_copy[maxlag - lg2 : maxlag - lg2 + len2]
                Ai = Ai['dummy TREASURY'].tolist()
                X0['lag2-'+str(lg2)] = Ai
            ar1 = Y_m.to_numpy()
            ar2 = X0.to_numpy()



        else:

            maxlag = lag1[i]


            Y_m = Y_copy[maxlag:]
            len2 = len(Y_m)

            X0 = X1_copy[maxlag - 0 : maxlag - 0 + len2]
            for lg1 in range(1,lag1[i]+1):
                Xi = X1_copy[maxlag - lg1 : maxlag - lg1 + len2]
                Xi  = Xi['dummy FTSE'].tolist()
                X0['lag1-'+str(lg1)] = Xi

            for lg2 in range(0,lag2[k]+1):
                Ai = X2_copy[maxlag - lg2 : maxlag - lg2 + len2]
                Ai = Ai['dummy TREASURY'].tolist()
                X0['lag2-'+str(lg2)] = Ai

            ar1 = Y_m.to_numpy()
            ar2 = X0.to_numpy()


        rf = RandomForestClassifier(n_estimators=500, max_features=0.25, criterion="entropy", class_weight="balanced")
        pip_baseline = make_pipeline(RobustScaler(), rf)
        results = cross_val_score(pip_baseline, ar2, ar1,
                             scoring="roc_auc", cv=5)
        res = 'Model '+str(i)+'.'+str(k) + ":  Mean Roc_Auc = "+str(results.mean()) + ',   lag FTSE = '+str(i)+ ',  lag TREASURY = '+str(k)
        print(res)

        if results.mean() >= max_ra:
            max_ra = results.mean()
            best_models3 = res

print('Best Model: ' + best_models3 )

Model 0.0:  Mean Roc_Auc = 0.6520463929146227,   lag FTSE = 0,  lag TREASURY = 0
Model 0.1:  Mean Roc_Auc = 0.6509028401238746,   lag FTSE = 0,  lag TREASURY = 1
Model 0.2:  Mean Roc_Auc = 0.6523755405508441,   lag FTSE = 0,  lag TREASURY = 2
Model 0.3:  Mean Roc_Auc = 0.6532398075087802,   lag FTSE = 0,  lag TREASURY = 3


KeyboardInterrupt: 

In [128]:
# Model: dummy FTSE ~ dummy E-MINI + dummy TREASURY

dep = ['dummy FTSE']
exog = ['dummy E-MINI','dummy TREASURY']
best_models4 = '0'

lag1 = [0,1,2,3,4,5,6,7,8,9,10]
lag2 = [0,1,2,3,4,5,6,7,8,9,10]

len1 = len(data[dep[0]])

X1 = data[exog[0]]
X1 = X1.to_frame()
X1 = X1.truncate(before=200000, after=250000)
X2 = data[exog[1]]
X2 = X2.to_frame()
X2 = X2.truncate(before=200000, after=250000)
Y = data[dep[0]]
Y = Y.to_frame()
Y = Y.truncate(before=200000, after=250000)
max_ra = 0

warnings.filterwarnings('ignore')

for i in range(len(lag1)):
    X1_copy = X1.copy()
    X2_copy = X2.copy()
    Y_copy = Y.copy()

    for k in range(len(lag2)):

        if lag2[k] >= lag1[i]:

            maxlag = lag2[k]

            Y_m = Y_copy[maxlag:]
            len2 = len(Y_m)

            X0 = X1_copy[maxlag - 0 : maxlag - 0 + len2]

            for lg1 in range(1,lag1[i]+1):
                Xi = X1_copy[maxlag - lg1 : maxlag - lg1 + len2]
                Xi  = Xi['dummy E-MINI'].tolist()
                X0['lag1-'+str(lg1)] = Xi

            for lg2 in range(0,maxlag+1):
                Ai = X2_copy[maxlag - lg2 : maxlag - lg2 + len2]
                Ai = Ai['dummy  TREASURY'].tolist()
                X0['lag2-'+str(lg2)] = Ai
            ar1 = Y_m.to_numpy()
            ar2 = X0.to_numpy()



        else:

            maxlag = lag1[i]

            Y_m = Y_copy[maxlag:]
            len2 = len(Y_m)

            X0 = X1_copy[maxlag - 0 : maxlag - 0 + len2]
            for lg1 in range(1,lag1[i]+1):
                Xi = X1_copy[maxlag - lg1 : maxlag - lg1 + len2]
                Xi  = Xi['dummy E-MINI'].tolist()
                X0['lag1-'+str(lg1)] = Xi

            for lg2 in range(0,lag2[k]+1):
                Ai = X2_copy[maxlag - lg2 : maxlag - lg2 + len2]
                Ai = Ai['dummy  TREASURY'].tolist()
                X0['lag2-'+str(lg2)] = Ai

            ar1 = Y_m.to_numpy()
            ar2 = X0.to_numpy()


        rf = RandomForestClassifier(n_estimators=500, max_features=0.25, criterion="entropy", class_weight="balanced")
        pip_baseline = make_pipeline(RobustScaler(), rf)
        results = cross_val_score(pip_baseline, ar2, ar1,
                             scoring="roc_auc", cv=5)
        res = 'Model '+str(i)+'.'+str(k) + ":  Mean Roc_Auc = "+str(results.mean()) + ',   lag E-MINI = '+str(i)+ ',  lag TREASURY = '+str(k)
        print(res)

        if results.mean() >= max_ra:
            max_ra = results.mean()
            best_models4 = res

print('Best Model: ' + best_models4 )




KeyError: 'dummy  FTSE'

In [ ]:
# Model: dummy TREASURY ~ dummy E-MINI + dummy FTSE

dep = ['dummy TREASURY']
exog = ['dummy E-MINI','dummy FTSE']
best_models5 = '0'

lag1 = [0,1,2,3,4,5,6,7,8,9,10]
lag2 = [0,1,2,3,4,5,6,7,8,9,10]

len1 = len(data[dep[0]])

X1 = data[exog[0]]
X1 = X1.to_frame()
X1 = X1.truncate(before=200000, after=250000)
X2 = data[exog[1]]
X2 = X2.to_frame()
X2 = X2.truncate(before=200000, after=250000)
Y = data[dep[0]]
Y = Y.to_frame()
Y = Y.truncate(before=200000, after=250000)
max_ra = 0

warnings.filterwarnings('ignore')

for i in range(len(lag1)):
    X1_copy = X1.copy()
    X2_copy = X2.copy()
    Y_copy = Y.copy()

    for k in range(len(lag2)):

        if lag2[k] >= lag1[i]:

            maxlag = lag2[k]

            Y_m = Y_copy[maxlag:]
            len2 = len(Y_m)

            X0 = X1_copy[maxlag - 0 : maxlag - 0 + len2]

            for lg1 in range(1,lag1[i]+1):
                Xi = X1_copy[maxlag - lg1 : maxlag - lg1 + len2]
                Xi  = Xi['dummy E-MINI'].tolist()
                X0['lag1-'+str(lg1)] = Xi

            for lg2 in range(0,maxlag+1):
                Ai = X2_copy[maxlag - lg2 : maxlag - lg2 + len2]
                Ai = Ai['dummy  FTSE'].tolist()
                X0['lag2-'+str(lg2)] = Ai
            ar1 = Y_m.to_numpy()
            ar2 = X0.to_numpy()



        else:

            maxlag = lag1[i]

            Y_m = Y_copy[maxlag:]
            len2 = len(Y_m)

            X0 = X1_copy[maxlag - 0 : maxlag - 0 + len2]
            for lg1 in range(1,lag1[i]+1):
                Xi = X1_copy[maxlag - lg1 : maxlag - lg1 + len2]
                Xi  = Xi['dummy E-MINI'].tolist()
                X0['lag1-'+str(lg1)] = Xi

            for lg2 in range(0,lag2[k]+1):
                Ai = X2_copy[maxlag - lg2 : maxlag - lg2 + len2]
                Ai = Ai['dummy  FTSE'].tolist()
                X0['lag2-'+str(lg2)] = Ai

            ar1 = Y_m.to_numpy()
            ar2 = X0.to_numpy()


        rf = RandomForestClassifier(n_estimators=500, max_features=0.25, criterion="entropy", class_weight="balanced")
        pip_baseline = make_pipeline(RobustScaler(), rf)
        results = cross_val_score(pip_baseline, ar2, ar1,
                             scoring="roc_auc", cv=5)
        res = 'Model '+str(i)+'.'+str(k) + ":  Mean Roc_Auc = "+str(results.mean()) + ',   lag E-MINI = '+str(i)+ ',  lag FTSE = '+str(k)
        print(res)

        if results.mean() >= max_ra:
            max_ra = results.mean()
            best_models4 = res

print('Best Model: ' + best_models5 )